# Initiate Feature Extraction

In [ ]:
import dotenv
import numpy as np
import os
import random
import mne
import enum


class PredCtChannelMapping(enum.Enum):
    Fp1 = 1
    Fp2 = 3
    F3 = 8
    F4 = 12
    C3 = 26
    C4 = 30
    P3 = 46
    P4 = 50
    O1 = 61
    O2 = 63
    F7 = 6
    F8 = 14
    T7 = 24 ## was T3
    T8 = 32 ## was T4
    P7 = 44 ## was T5
    P8 = 52 ## was T6

class ModmaChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

dotenv.load_dotenv()
modmaEpochsDir = os.getenv("MODMA_10S_EPOCHS_DIR")
predctEpochsDir = os.getenv("PREDCT_10S_EPOCHS_DIR")
modmaFeaturesDir = os.getenv("MODMA_FEATURES_DIR")
predctFeaturesDir = os.getenv("PREDCT_FEATURES_DIR")


# === CHANGE THIS TO SWITCH BETWEEN MODMA AND PREDCT DATASET ===
whichEpochsDir = modmaEpochsDir
whichFeaturesDir = modmaFeaturesDir

# --- SET VARIABLES FOR FEATURES EXTRACTION PROCESS ---
NUM_CHANNELS = 16
SFREQ = 250
NUM_UNSEEN_HC_SUBJECTS = 2
NUM_UNSEEN_MDD_SUBJECTS = 2
# ===============================================================

mddEpochDir = whichEpochsDir + "/mdd"
hcEpochDir = whichEpochsDir + "/hc"

In [ ]:
mddFiles = [os.path.join(mddEpochDir, f) for f in os.listdir(mddEpochDir) if f.endswith(".fif")]
hcFiles = [os.path.join(hcEpochDir, f) for f in os.listdir(hcEpochDir) if f.endswith(".fif")]
totalMddSubjects = len(mddFiles)
totalHcSubjects = len(hcFiles)
numMddEpochs = max(0, totalMddSubjects - NUM_UNSEEN_MDD_SUBJECTS)
numHcEpochs = max(0, totalHcSubjects - NUM_UNSEEN_HC_SUBJECTS)

mddUnseenFiles = random.sample(mddFiles, NUM_UNSEEN_MDD_SUBJECTS)
hcUnseenFiles = random.sample(hcFiles, NUM_UNSEEN_HC_SUBJECTS)

mddEpochFiles = [f for f in mddFiles if f not in mddUnseenFiles][:numMddEpochs]
hcEpochFiles = [f for f in hcFiles if f not in hcUnseenFiles][:numHcEpochs]

mddUnseenEpochs = mne.concatenate_epochs([mne.read_epochs(f) for f in mddUnseenFiles])
hcUnseenEpochs = mne.concatenate_epochs([mne.read_epochs(f) for f in hcUnseenFiles])
mddEpochs = mne.concatenate_epochs([mne.read_epochs(f) for f in mddEpochFiles])
hcEpochs = mne.concatenate_epochs([mne.read_epochs(f) for f in hcEpochFiles])

In [11]:
print(f"Total MDD subjects: {totalMddSubjects}")
print(f"Total HC subjects: {totalHcSubjects}")
print(f"Number of unseen MDD subjects: {len(mddUnseenFiles)}")
print(f"Number of unseen HC subjects: {len(hcUnseenFiles)}")
print(f"Number of MDD epochs: {len(mddEpochFiles)}")
print(f"Number of HC epochs: {len(hcEpochFiles)}")

print(f"mddUnseenEpochs shape: {mddUnseenEpochs.get_data().shape}")
print(f"hcUnseenEpochs shape: {hcUnseenEpochs.get_data().shape}")
print(f"mddEpochs shape: {mddEpochs.get_data().shape}")
print(f"hcEpochs shape: {hcEpochs.get_data().shape}")

Total MDD subjects: 24
Total HC subjects: 29
Number of unseen MDD subjects: 2
Number of unseen HC subjects: 2
Number of MDD epochs: 22
Number of HC epochs: 27
mddUnseenEpochs shape: (31, 16, 2500)
hcUnseenEpochs shape: (31, 16, 2500)
mddEpochs shape: (330, 16, 2500)
hcEpochs shape: (406, 16, 2500)


## Feature Extraction

### 1. Band Power

In [29]:
from brainmdd.features.psd import EEGPowerSpectrum

# NOTE: featureDict looks like this:
featureDict = {
    "band_power": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    }

resultFileEpochMap = [
    ("bandpower_MDD.npy", mddEpochs.get_data()),
    ("bandpower_HC.npy", hcEpochs.get_data()),
    ("bandpower_MDD_unseen.npy", mddUnseenEpochs.get_data()),
    ("bandpower_HC_unseen.npy", hcUnseenEpochs.get_data())
]

bandPowerFeatureDir = whichFeaturesDir + "/bandpower"
if not os.path.exists(bandPowerFeatureDir):
    os.makedirs(bandPowerFeatureDir)    

In [30]:
def calculate_and_save_bandpower_features(result_file_epoch_map, bandpower_feature_dir, sfreq):
    for filename, epochs in result_file_epoch_map:        
        psdObj = EEGPowerSpectrum(epochs, sfreq, ifNormalize=False)
        bandpowerFeatures = psdObj.run()
        save_path = os.path.join(bandpower_feature_dir, filename)
        np.save(save_path, bandpowerFeatures)
        print(f"Saved {filename} to {save_path}")
    print("Feature extraction and saving completed.")

In [31]:
calculate_and_save_bandpower_features(resultFileEpochMap, bandPowerFeatureDir, SFREQ)

Saved bandpower_MDD.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/bandpower/bandpower_MDD.npy
Saved bandpower_HC.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/bandpower/bandpower_HC.npy
Saved bandpower_MDD_unseen.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/bandpower/bandpower_MDD_unseen.npy
Saved bandpower_HC_unseen.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/bandpower/bandpower_HC_unseen.npy
Feature extraction and saving completed.


### 2. Relative Power

In [32]:
from brainmdd.features.psd import EEGPowerSpectrum


# NOTE: featureDict looks like this:
featureDict = {
    "relative_power": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    }

resultFileEpochMap = [
    ("relative_power_MDD.npy", mddEpochs.get_data()),
    ("relative_power_HC.npy", hcEpochs.get_data()),
    ("relative_power_MDD_unseen.npy", mddUnseenEpochs.get_data()),
    ("relative_power_HC_unseen.npy", hcUnseenEpochs.get_data())
]

relativePowerFeatureDir = whichFeaturesDir + "/relativepower"
if not os.path.exists(relativePowerFeatureDir):
    os.makedirs(relativePowerFeatureDir)

In [33]:
def calculate_and_save_relative_power_features(result_file_epoch_map, relative_power_feature_dir, sfreq):
    for filename, epochs in result_file_epoch_map:
        psdObj = EEGPowerSpectrum(epochs, sfreq, ifNormalize=True)  
        relativePowerFeatures = psdObj.run()
        save_path = os.path.join(relative_power_feature_dir, filename)
        np.save(save_path, relativePowerFeatures)
        print(f"Saved {filename} to {save_path}")
    print("Relative power feature extraction and saving completed.")

In [34]:
calculate_and_save_relative_power_features(resultFileEpochMap, relativePowerFeatureDir, SFREQ)

Saved relative_power_MDD.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/relativepower/relative_power_MDD.npy
Saved relative_power_HC.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/relativepower/relative_power_HC.npy
Saved relative_power_MDD_unseen.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/relativepower/relative_power_MDD_unseen.npy
Saved relative_power_HC_unseen.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/relativepower/relative_power_HC_unseen.npy
Relative power feature extraction and saving completed.


### 3. Higuchi Fractral Dimension (HFD)

In [26]:
from brainmdd.features.hfd import EEGHiguchiFractalDimension

# NOTE: featureDict looks like this:
featureDict = {
    "HFD": {
        "mdd": [],
        "control": []
    }
}

resultFileEpochMap = [
    ("hfd_MDD.npy", mddEpochs.get_data()),
    ("hfd_HC.npy", hcEpochs.get_data()),
    ("hfd_unseen_MDD.npy", mddUnseenEpochs.get_data()),
    ("hfd_unseen_HC.npy", hcUnseenEpochs.get_data())
]

hfdFeatureDir = whichFeaturesDir + "/hfd"
if not os.path.exists(hfdFeatureDir):
    os.makedirs(hfdFeatureDir)

In [27]:
def calculate_and_save_hfd_features(result_file_epoch_map, hfd_feature_dir):
    for filename, epochs in result_file_epoch_map: 
        hfdObj = EEGHiguchiFractalDimension(epochs)
        hfdFeatures = hfdObj.run()
        save_path = os.path.join(hfd_feature_dir, filename)
        np.save(save_path, hfdFeatures)
        print(f"Saved {filename} to {save_path}")
    print("HFD feature extraction and saving completed.")

In [28]:
calculate_and_save_hfd_features(resultFileEpochMap, hfdFeatureDir)

Saved hfd_MDD.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/hfd/hfd_MDD.npy
Saved hfd_HC.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/hfd/hfd_HC.npy
Saved hfd_unseen_MDD.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/hfd/hfd_unseen_MDD.npy
Saved hfd_unseen_HC.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/hfd/hfd_unseen_HC.npy
HFD feature extraction and saving completed.


### 4. Coherences

In [35]:
from brainmdd.features.coherence import EEGCoherence

# NOTE: featureDict looks like this:
featureDict = {
    "coherence": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    } 

resultFileEpochMap = [
    ("coherence_MDD.npy", mddEpochs.get_data()),
    ("coherence_HC.npy", hcEpochs.get_data()),
    ("coherence_unseen_MDD.npy", mddUnseenEpochs.get_data()),
    ("coherence_unseen_HC.npy", hcUnseenEpochs.get_data())
]

coherenceFeatureDir = whichFeaturesDir + "/coherence"
if not os.path.exists(coherenceFeatureDir):
    os.makedirs(coherenceFeatureDir)

In [36]:
def calculate_and_save_coherence_features(result_file_epoch_map, coherence_feature_dir, sfreq):
    for filename, epoch_data in result_file_epoch_map:
        coherenceObj = EEGCoherence(epoch_data, sfreq)
        coherenceFeatures = coherenceObj.run()
        save_path = os.path.join(coherence_feature_dir, filename)
        np.save(save_path, coherenceFeatures)
        print(f"Saved {filename} to {save_path}")
    print("Coherence feature extraction and saving completed.")

In [37]:
calculate_and_save_coherence_features(resultFileEpochMap, coherenceFeatureDir, SFREQ)

Saved coherence_MDD.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/coherence/coherence_MDD.npy
Saved coherence_HC.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/coherence/coherence_HC.npy
Saved coherence_unseen_MDD.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/coherence/coherence_unseen_MDD.npy
Saved coherence_unseen_HC.npy to /Users/wachiii/Workschii/brain-mdd/data/features/modma/coherence/coherence_unseen_HC.npy
Coherence feature extraction and saving completed.


### 5. Correlation Dimension (CD)

In [38]:
from brainmdd.features.correlationdimension import EEGCorrelationDimension

# NOTE: featureDict looks like this:
featureDict = {
    "CD": {
        "mdd": [],
        "control": []
    }
}

resultFileEpochMap = [
    ("cd_MDD.npy", mddEpochs.get_data()),
    ("cd_HC.npy", hcEpochs.get_data()),
    ("cd_unseen_MDD.npy", mddUnseenEpochs.get_data()),
    ("cd_unseen_HC.npy", hcUnseenEpochs.get_data())
]

cdFeatureDir = whichFeaturesDir + "/cd"
if not os.path.exists(cdFeatureDir):
    os.makedirs(cdFeatureDir)

In [39]:
def calculate_and_save_cd_features(result_file_epoch_map, cd_feature_dir, sfreq):
    for filename, epoch_data in result_file_epoch_map:
        cdObj = EEGCorrelationDimension(epoch_data, sfreq)
        cdFeatures = cdObj.run()
        save_path = os.path.join(cd_feature_dir, filename)
        np.save(save_path, cdFeatures)
        print(f"Saved {filename} to {save_path}")
    print("Coherence Density feature extraction and saving completed.")

In [ ]:
calculate_and_save_cd_features(resultFileEpochMap, cdFeatureDir, SFREQ)